In [231]:
import numpy as np
#filename = "example_15.txt"
#filename = "example_15_2.txt"
filename = "input_15"
with open(filename) as f:
    input = f.read().split("\n\n")
input_grid = np.asarray([[i for i in line] for line in input[0].split("\n")])
movements = "".join(input[1].split("\n"))

## Part 1:

In [232]:
def update_grid(grid, robot_coordinate, movement):
    # get direction from movement:
    if movement == "v":
        direction = np.array([1,0])
    elif movement == ">":
        direction = np.array([0,1])
    elif movement == "^":
        direction = np.array([-1,0])
    elif movement == "<":
        direction = np.array([0,-1])
    else:
        ValueError

    if grid[robot_coordinate] != "@":
        ValueError

    #now go in the direction, save box-coordinates until wall or empty space
    boxes_to_move = []
    for i in range(1, len(grid)):
        new_coordinate = tuple(np.array(robot_coordinate) + i*direction)
        if grid[new_coordinate] == "O":
            boxes_to_move.append(np.array(new_coordinate))
        elif grid[new_coordinate] == "#":
            return grid, robot_coordinate
        else:
            break
        
    for box in boxes_to_move:
            grid[tuple(box + direction)] = "O"
    new_robot = tuple(np.array(robot_coordinate) + direction)
    grid[new_robot] = "@"
    grid[robot_coordinate] = "."
    return grid, new_robot

In [233]:
# find initial robot coordinate:
robot_initial = tuple(i.item() for i in np.array(np.where(input_grid == "@")))

In [234]:
grid = input_grid.copy()
for movement in movements:
    grid, robot_initial = update_grid(grid, robot_initial, movement)

In [235]:
def score(grid, item):
    weight = np.array((100,1))
    GPS_coords = np.array(weight) * np.array(np.where(grid == item)).T
    return np.sum(GPS_coords)

In [ ]:
score(grid, "O").item()

## Part 2:

In [237]:
grid = input_grid.copy()
# enlarge grid:
new_grid = []
for line in grid:
    new_line = []
    for el in line:
        if el == "#":
            new_line += ["#","#"]
        elif el == "O":
            new_line += ["[","]"]
        elif el == ".":
            new_line += [".","."]
        else:
            new_line += ["@","."]
    new_grid.append(new_line)
grid = np.array(new_grid)

In [ ]:
def print_grid(grid):
    for line in grid:
        print("".join(line))
print_grid(grid)

In [239]:
def update_grid(grid, robot_coordinate, movement):
    # get direction from movement:
    directions = {"v":(1,0),">":(0,1),"^":(-1,0),"<":(0,-1)}
    direction = np.array(directions[movement])

    boxes_to_move = {}
    positions_to_check = [tuple(np.array(robot_coordinate) + direction)]
    while len(positions_to_check) > 0:
        position_to_check = positions_to_check.pop()
        for i in range(len(grid)):
            new_coordinate = tuple(np.array(position_to_check) + i*direction)
            item = grid[new_coordinate]
            if item == "#":
                return grid, robot_coordinate
            elif item in ("[","]") and movement in ("^","v"):
                boxes_to_move[new_coordinate] = item
                bracket = {"[": 1, "]": -1}
                partner = (new_coordinate[0], new_coordinate[1] + bracket[item])
                if partner not in boxes_to_move.keys():
                    positions_to_check.append(partner)
            elif item in ("[","]") and movement in ("<",">"):
                boxes_to_move[new_coordinate] = item
            else:
                break

    for box in boxes_to_move.keys():
        grid[box] = "."
    for box in boxes_to_move.keys():
        grid[tuple(np.array(box) + direction)] = boxes_to_move[box]
        
    new_robot = tuple(np.array(robot_coordinate) + direction)
    grid[new_robot] = "@"
    grid[robot_coordinate] = "."

    return grid, new_robot

In [ ]:
robot_initial = tuple(i.item() for i in np.array(np.where(grid == "@")))
for movement in movements:
    grid, robot_initial = update_grid(grid, robot_initial, movement)
print_grid(grid)
score(grid, "[").item()